### • 합성곱 신경망 (CNN)

: 이미지 인식과 음성 인식 등 다양한 곳에서 사용됨  →  특히 이미지 인식 분야에서 딥러닝 활용한 기법 거의 다 CNN 기초

• 합성곱 계층은 형상 유지 / 이미지도 3차원 데이터로 입력 받고, 다음 계층에도 3차원 데이터도 전달

### • 완전 연결 

: 신경망은 인접하는 계층의 모든 뉴런과 결합

→ Affine 계층 ( 완전히 연결된 계층 )으로 구현

##### ▶︎ 완전연결 계층의 문제점

• 완전연결 계층은 인접하는 계층의 뉴런이 모두 연결되고 출력 수는 임의로 정할 수 있었다.

→ 데이터의 형상이 무시된다 / 모든 입력 데이터를 동등한 뉴런 (같은 차원의 뉴런)으로 취급하여 형상에 담긴 정보를 살릴 수 없다. 

### • 패딩 

: 합성곱 연산을 수행하기 전에 입력 데이터 주변을 특정 값 (예컨대 0)으로 채우기도 함.

→ 주로 출력 크기를 조정할 목적으로 사용


### • 스트라이드

: 필터 적용하는 위치의 간격

##### • 3차원 합성곱 연산 ( 데이터와 필터 - 직육면체 블록 )

► 주의할 점

: 입력 데이터의 채널 수와 필터의 채널 수가 같아야 한다. 

( 필터 자체의 크기 원하는 값으로 설정 가능. 다만, 모든 채널의 필터 크기가 같아야함.)

### • 배치 처리 

• 신경망 처리에서는 입력 데이터를 한 덩어리로 묶어 배치로 처리

→ 완전연결 신경망을 구현하면서 이 방식을 지원하여 처리 효율 높이고, 미니배치 방식의 학습도 지원

### • 풀링 계층 

: 세로, 가로 방향의 공간을 줄이는 연산 

▶︎ 특징

1. 학습해야 할 매개변수가 없다.

: 풀링은 대상 영역에서 최댓값이나 평균 취하는 명확한 처리이므로 

2. 채널 수 변하지 않는다.

: 입력 데이터의 채널 수 그대로 출력 데이터로 내보낸다.


3. 입력 변화에 영향을 적게 받는다 (강건하다)

: 입력 데이터가 조금 변해도 풀링의 결과는 잘 변하지 않는다. 


In [3]:
# 4차원 배열 구현 
import numpy as np

x = np.random.rand(10, 1, 28, 28) # 무작위로 데이터 생성
x.shape

(10, 1, 28, 28)

In [6]:
print(x[0].shape) 
print(x[1].shape)

# CNN은 4차원 데이터 다룬다. 합성곱 연산 구현이 복잡해질 것 같지만, im2col이라는 '트릭'이 문제를 단순하게 만든다.

(1, 28, 28)
(1, 28, 28)


### • im2col 

: 입력 데이터를 필터링(가중치 계산)하기 좋게 전개하는(펼치는) 함수 

###
→ 3차원 입력 데이터에 im2col 적용하면 2차원 행렬로 바뀜 

(정확히는 배치 안의 데이터 수까지 포함한 4차원 데이터 2차원으로 변환)


#### 
▶︎ CNN은 데이터를 4차원 배열로 저장하므로 2차원인 출력 데이터를 4차원으로 변형 (합성곱 계층의 구현 흐름) 

In [8]:
# im2col 함수 구현

def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """다수의 이미지를 입력받아 2차원 배열로 변환한다(평탄화).
    
    Parameters
    ----------
    input_data : 4차원 배열 형태의 입력 데이터(이미지 수, 채널 수, 높이, 너비)
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩 """


In [11]:
import sys, os
sys.path.append('/Users/krc/Downloads/deep-learning-from-scratch-master')
from common.util import im2col

x1 = np.random.rand(1, 3, 7, 7) # (데이터 수, 채널 수, 높이, 너비)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

x2 = np.random.rand(10, 3, 7, 7) # (데이터 10개)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(9, 75)
(90, 75)


In [12]:
class Convolution:
    def __init__(self, w, b, stride=1, pad=0):
        self.w = w
        self.b = b
        self.stride = stride
        self.pad = pad
    
    def forward(self, x):
        FN, C, FH, FW = self.w.shape
        N, C, H, W = x.shape
        out_h = int( 1+ (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + ( W + 2*self.pad - FW) / self.stride) 

        # 중요한 부분 
        # → 입력 데이터를 im2col로 전개하고 필터도 reshape을 사용하여 2차원 배열로 전개 후 전개한 두 행렬의 곱 구함 
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_w  = self.w.reshape(FN, -1).T  # 필터 전개 / reshape 두번째 인수 -1로 지정했는데 reshape이 제공하는 편의 기능
        out = np.dot(co1, col_w) + self.b

        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2) # 출력 데이터를 적절한 형상으로 바꾸어 준다.

        return out    

# reshape에 -1 지정하면 다차원 배열 원소 수가 변환 후에도 똑같이 유지되게 적절히 묶어준다. 
# 앞 코드에서 (10, 3, 5, 5) 형상한 다차원 배열 w 원소 수는 총 750개
# 이 배열에 reshape(10, -1) 호출하면 750개 원소를 10묶음으로, 즉 형상이 (10,75)인 배열 만들어준다. 

In [13]:
# 풀링 계층 구현 

class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
        self.x = None
        self.arg_max = None

    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)

        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)

        arg_max = np.argmax(col, axis=1)
        out = np.max(col, axis=1)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

        self.x = x
        self.arg_max = arg_max

        return out

    def backward(self, dout):
        dout = dout.transpose(0, 2, 3, 1)
        
        pool_size = self.pool_h * self.pool_w
        dmax = np.zeros((dout.size, pool_size))
        dmax[np.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()
        dmax = dmax.reshape(dout.shape + (pool_size,)) 
        
        dcol = dmax.reshape(dmax.shape[0] * dmax.shape[1] * dmax.shape[2], -1)
        dx = col2im(dcol, self.x.shape, self.pool_h, self.pool_w, self.stride, self.pad)
        
        return dx


### • 풀링 계층 구현 

1. 입력 데이터 전개한다.

2. 행별 최댓값 구한다.

3. 적절한 모양으로 성형한다.

####
• 2차원 배열, 즉 행렬이라면 axis=0은 열 방향, axis=1은 행 방향